In [1]:
# pip install pandas numpy matplotlib pillow mxnet onnx onnxruntime

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import os
import json
from PIL import Image
import onnxruntime as ort
import onnx
import requests
from io import BytesIO
import json

%matplotlib inline

In [3]:
async def preprocess(image):
    # resize so that the shorter side is 256, maintaining aspect ratio
    def image_resize(image, min_len):
        image = Image.fromarray(image)
        ratio = float(min_len) / min(image.size[0], image.size[1])
        if image.size[0] > image.size[1]:
            new_size = (int(round(ratio * image.size[0])), min_len)
        else:
            new_size = (min_len, int(round(ratio * image.size[1])))
        image = image.resize(new_size, Image.BILINEAR)
        return np.array(image)
    image = image_resize(image, 256)

    # Crop centered window 224x224
    def crop_center(image, crop_w, crop_h):
        h, w, c = image.shape
        start_x = w//2 - crop_w//2
        start_y = h//2 - crop_h//2
        return image[start_y:start_y+crop_h, start_x:start_x+crop_w, :]
    image = crop_center(image, 224, 224)

    # transpose
    image = image.transpose(2, 0, 1)

    # convert the input data into the float32 input
    img_data = image.astype('float32')

    # normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]

    # add batch channel
    norm_img_data = norm_img_data.reshape(1, 3, 224, 224).astype('float32')
    return norm_img_data

async def get_tensor_from_batch(image_size, img_array, array_expected):
    k = 0
    #print(len(img_array))
    count_images = len(img_array)
    array_expected[0] = count_images
    
    # Create a NumPy array to hold the processed data
    process_array = np.empty((count_images, 3, image_size, image_size), dtype=np.float32)
    
    for i in range(count_images):
        element = img_array[i]
        data = np.asarray(element)
        preprocessed_data = await preprocess(data)
        process_array[k, :, :, :] = preprocessed_data
        k += 1

    return process_array


def get_images_array(url_array):
    # Send a GET request to the URL
    if len(url_array) == 0:
        response = requests.get("https://picsum.photos/v2/list")
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()

            # Initialize an empty list to store the images
            images = []

            # Loop through the data and download/save the images
            for item in data:
                download_url = item["download_url"]

                # Send a GET request to the image URL
                image_response = requests.get(download_url)

                # Check if the image request was successful
                if image_response.status_code == 200:
                    # Open the image using PIL
                    img = Image.open(BytesIO(image_response.content))

                    # Append the image to the list
                    images.append(img)

                    # Optionally, save the image to a file
                    # img.save(f"{item['id']}.jpg")

            return images
            # Now, you have a list of PIL Image objects in the 'images' variable
        else:
            print("Failed to fetch data from the URL")
    else:
        # Initialize an empty list to store the images
        images = []
        for url in url_array:
            image_response = requests.get(url)
            # Check if the request was successful
            if image_response.status_code == 200:
                img = Image.open(BytesIO(image_response.content))

                # Append the image to the list
                images.append(img)

                # Optionally, save the image to a file
                # img.save(f"{item['id']}.jpg")
            else:
                print("Failed to fetch data from the URL")
        return images
        # Now, you have a list of PIL Image objects in the 'images' variable
        

    # You can access the images from the 'images' list and manipulate them as needed
# Post-processing function for ImageNet models


def download(content, file_name, content_type):
    with open(file_name, 'wb') as file:
        file.write(content)

def on_download(json_data, file_name):
    json_str = json.dumps(json_data)
    download(json_str.encode('utf-8'), file_name, 'text/plain')

In [4]:

# General function to run the ONNX model with a given input tensor
async def run_onnx_model(tensor):
    # Initialize the ONNX Runtime session
    options = ort.SessionOptions()
    options.intra_op_num_threads = 1
    options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
    options.add_session_config_entry('session.intra_op_thread_affinities', '1')
    options.enable_profiling=False
    ort_session = ort.InferenceSession(
            './model.onnx',
            sess_options=options,
            providers=['CPUExecutionProvider'])
    input_name = ort_session.get_inputs()[0].name
    feeds = {input_name: tensor}

    # Run the model with the input tensor and get the result
    result = ort_session.run(None, feeds)

    return result


In [5]:
import time
import requests

# Function to run the ONNX model with a batch of images from URLs
async def run_batch_model(image_size, array_expected, url_array, images_array, model_name):
    times = {}
    start_time = time.time()
    time_image_array = time.time()
    if len(url_array) == 0:
        url_array = url_array = ["https://i.imgur.com/b0rgmfl.jpg",
                        "https://i.imgur.com/T0wfmza.jpg",
                        "https://i.imgur.com/pX4HIwE.jpg",
                        "https://i.imgur.com/Mh0CzBL.jpg",
                        "https://i.imgur.com/ShJfWWk.jpg"]
    

    print("Getting images")
    if len(images_array) == 0:
        images_array = get_images_array(url_array)

    print("Obtained array:")
    print(images_array)

    time_image_array_elapsed = time.time() - time_image_array
    times["images"] = time_image_array_elapsed
    print("Time spent fetching images: ", time_image_array_elapsed)

    print("Getting tensor")
    time_tensor = time.time()
    tensor_images = await get_tensor_from_batch(image_size, images_array, array_expected)
    time_tensor_elapsed = time.time() - time_tensor
    times["memory"] = tensor_images.nbytes
    times["tensor"] = time_tensor_elapsed
    print("Time spent creating tensor from images: ", time_tensor_elapsed)

    print("Running model")
    time_run_model = time.time()
    result = await run_onnx_model(tensor_images)
    time_run_model_elapsed = time.time() - time_run_model
    times["model"] = time_run_model_elapsed
    print("Time spent running model: ", time_run_model_elapsed)

    finish_time = time.time() - start_time
    times["total"] = finish_time
    print("Total processing time: ", finish_time)
    #download_name = model_name + "-" + "Data-Python" + ".json"
    #on_download(json.dumps(result), download_name)
    return times

# Define the missing functions get_images_array, get_tensor_from_batch, run_onnx_model, and post_json
# These functions will depend on your specific implementation.


In [6]:
import asyncio
import time

# Function to run benchmark
async def run_benchmark(image_size, array_expected, url, urls_array, model_name):
    pre_images_array = get_images_array(urls_array)
    await run_batch_model(image_size, array_expected, urls_array,pre_images_array, model_name)
    
    for i in range(1,11):
        # Create arrays that will store the JSON data
        times_json = []
        times_json_avg = []
        
        # Fetch images based on the URLs
        images_array = get_images_array(urls_array)
        
        # Start the repetitions
        for rep in range(1, len(images_array) + 1):
            if rep == 21:
                break
            pre_images_array = images_array[:rep]
            print("Testing with:", len(pre_images_array), "images")
            
            # Create JSON for each repetition
            times = await run_batch_model(image_size, array_expected, urls_array, pre_images_array, model_name)
            times_json.append(times)
        
        # Save JSON data to a file for processing
        print("Json Times:", times_json)
        
        # Download JSON data
        download_name = model_name + "-" + "python" + str(i) + ".json"
        #on_download(json.dumps(times_json), download_name)
        print("Download ready")
    
    # Calculate and save average times if needed
    on_download(times_json_avg, "avg-" + model_name + "-" + backend + ".json")


In [7]:
result = await run_batch_model(224, [1,3,224,224],'',[],"resnet")

Getting images
Obtained array:
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1200x811 at 0x189FF1ABF90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1300x866 at 0x189E5FC9150>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=637x596 at 0x189FFCFF150>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x750 at 0x189E5FA3590>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=800x546 at 0x189FF081A50>]
Time spent fetching images:  6.983261346817017
Getting tensor
Time spent creating tensor from images:  0.10169816017150879
Running model
Time spent running model:  1.6638109683990479
Total processing time:  8.748770475387573


In [8]:
await run_benchmark(224, [1,3,224,224],'',[],"resnet")

: 

In [ ]:
def softmax(x):
    """Compute softmax values for each set of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)

def postprocess(scores):
    '''
    Postprocessing without mxnet gluon
    The function takes scores generated by the network and returns the class IDs in decreasing order
    of probability using numpy
    '''
    
    prob = softmax(scores)
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    return a

In [ ]:
import numpy as np
import json

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def postprocess2():
    # Load labels
    with open('labels.txt', 'r') as f:
        labels = [line.strip() for line in f]

    # Load output data
    with open('output.json', 'r') as f:
        data = json.load(f)
        # Convert the score data from dictionary to numpy array
        scores = np.array(list(data['data'].values()))

    # Apply softmax to the scores
    probabilities = softmax(scores)

    # Get the indices of the scores sorted by probability
    sorted_indices = np.argsort(probabilities)[::-1]

    # Print the top prediction along with its probability
    top_prediction = sorted_indices[0]
    print(f"Class: {labels[top_prediction]}, Probability: {probabilities[top_prediction]}")

postprocess2()


Class: n03832673 notebook, notebook computer, Probability: 0.776560417188784


In [ ]:
# Import JSON module
import json
 # Opening JSON file
with open('output.json') as json_file:
    result = json.load(json_file)
    
arrayResult = result['data'].map()


AttributeError: 'dict' object has no attribute 'map'

In [ ]:
post = postprocess(list(result['data'].values()))
post